From https://pub.towardsai.net/text-classification-with-simple-transformers-a29d13358135

In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from torch.utils.tensorboard import SummaryWriter

In [ ]:
os.environ['TOKENIZERS_PARALLELISM'] =  'True'
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
train.drop(['id', 'keyword', 'location'], axis=1, inplace=True)
test.drop(['id', 'keyword', 'location'], axis=1, inplace=True)
train.columns = ['text', 'labels']
train_df, valid_df = train_test_split(train, test_size=0.2, stratify=train['labels'], random_state=42)

In [ ]:
model_args = ClassificationArgs(num_train_epochs=3, overwrite_output_dir=True, manual_seed=42, best_model_dir='./outputs_bert/', evaluate_during_training=True, wandb_project='disaster_tweets_bert')
model = ClassificationModel(model_type='bert', model_name='bert-base-cased', use_cuda=False, num_labels=2, args=model_args)

In [ ]:
model.train_model(train_df, eval_df=valid_df)

In [ ]:
result, model_outputs, wrong_preds = model.eval_model(valid_df)
predictions = np.argmax(model_outputs, axis=-1)
print('f1 score:', f1_score(valid_df['labels'], predictions), 'accuracy', accuracy_score(valid_df['labels'], predictions))

In [ ]:
loaded_model = ClassificationModel("bert", "./outputs_bert/", use_cuda=False)
result, model_outputs, wrong_preds = loaded_model.eval_model(valid_df)
predictions = np.argmax(model_outputs, axis=-1)
print('f1 score:', f1_score(valid_df['labels'], predictions), 'accuracy', accuracy_score(valid_df['labels'], predictions))

In [ ]:
result, model_outputs, wrong_preds = loaded_model.eval_model(train_df)
predictions = np.argmax(model_outputs, axis=-1)
print('f1 score:', f1_score(train_df['labels'], predictions), 'accuracy', accuracy_score(train_df['labels'], predictions))

In [ ]:
loaded_model = ClassificationModel("bert", "./outputs_bert/", use_cuda=False)
writer = SummaryWriter("./")
writer.add_graph(loaded_model, valid_df)
writer.close()

In [ ]:
from torchviz import make_dot
result, model_outputs, wrong_preds = loaded_model.eval_model(valid_df)
y = np.argmax(model_outputs, axis=-1)

In [ ]:
make_dot(model_outputs.mean(), params=dict(loaded_model.model.named_parameters()))

In [ ]:
dict(loaded_model.model.named_parameters()).size()